# Reset properties in NetCDF mapfile
* Velocities
* Concentrations
* Turbulent energy and dissipation
* Vertical and horizontal eddy viscosity and diffusivity
* Transport layer composition

In [1]:
import xarray as xr
from JulesD3D.processNetCDF import fixMeshGrid, addUnderlayerCoords, makeVelocity
import numpy as np
from IPython.display import Markdown as md

In [2]:
filename = '/Users/julesblom/ThesisResults/Slope1.00/trim-36km_200m_W60ChannelRun10_compressed.nc' # Puts all of DP_BEDLYR at 0.5 m?, which is the height of the transport layer

trim = xr.open_dataset(filename)

In [3]:
new_write_filename = filename[0:-14] + '.nc'
new_write_filename

'/Users/julesblom/ThesisResults/Slope1.00/trim-36km_200m_W60ChannelRun10.nc'

In [ ]:
sand = 0 
silt = 1
last_timestep = -1

# this has happened
if silt == sand:
    raise Exception("You stupid")

In [ ]:
interfaces_shape = trim.W.isel(time=last_timestep).shape
centers_shape = trim.V1.isel(time=last_timestep).shape

In [ ]:
all_ones_centers = np.ones(centers_shape)

all_zeros_centers = np.zeros(centers_shape)
all_zeros_interfaces = np.zeros(interfaces_shape)

## Reset velocity

In [ ]:
trim['V1'][last_timestep] = all_zeros_centers      # Horizontal velocity U component
trim['U1'][last_timestep] = all_zeros_centers      # Horizontal velocity U component
trim['WPHY'][last_timestep] = all_zeros_centers    # Vertical velocity component?
trim['W'][last_timestep] = all_zeros_interfaces    # Velocity angle?

## Reset Eddies

In [ ]:
trim['VICWW'][last_timestep] = all_zeros_interfaces # Vertical eddy viscosity-3D
trim['DICWW'][last_timestep] = all_zeros_interfaces # Vertical eddy diffusivity-3D
trim['VICUV'][last_timestep] = all_zeros_centers    # Horizontal eddy viscosity

In [ ]:
trim.VICUV.isel(time=0).min()

## Reset Water Level

In [ ]:
water_level_shape = trim.S1.isel(time=0).shape
reset_water_level = np.zeros(water_level_shape)

trim['S1'][last_timestep] = reset_water_level

## Reset concentrations

In [ ]:
test_conc_shape = trim.R1.isel(time=last_timestep).shape
reset_concentrations = np.zeros(test_conc_shape)

In [ ]:
trim['R1'][last_timestep] = reset_concentrations

## Reset density

In [ ]:
initial_density = float(trim.RHOCONST.values)
reset_densities = all_ones_centers * initial_density

trim['RHO'][last_timestep] = reset_densities

## Reset transport layer

### Volume fraction

In [ ]:
vol_frac_sand_transport_layer_end = trim.LYRFRAC.isel(time=last_timestep, LSEDTOT=sand, nlyr=0)
vol_frac_silt_transport_layer_end = trim.LYRFRAC.isel(time=last_timestep, LSEDTOT=silt, nlyr=0)

In [ ]:
# Replace transport layer with 50%/50% composition
replaced_vol_frac_sand = vol_frac_sand_transport_layer_end.where(vol_frac_sand_transport_layer_end.values == 0, 0.5)
replaced_vol_frac_silt = vol_frac_silt_transport_layer_end.where(vol_frac_silt_transport_layer_end.values == 0, 0.5)

In [ ]:
# Reset volume composition in transport layer at final timestep
trim.LYRFRAC[last_timestep, sand, 0] = replaced_vol_frac_sand.values
trim.LYRFRAC[last_timestep, silt, 0] = replaced_vol_frac_silt.values

### Mass of sediment

In [ ]:
# for 50 cm thick transport layer!
initial_sand_mass = 400 # 16000 # [kg/m2]
initial_silt_mass = 125 # 5000  # [kg/m2]

In [ ]:
mass_sand_transport_layer_end = trim.MSED.isel(time=last_timestep, LSEDTOT=sand, nlyr=0)
mass_silt_transport_layer_end = trim.MSED.isel(time=last_timestep, LSEDTOT=silt, nlyr=0)

replaced_mass_sand = mass_sand_transport_layer_end.where(mass_sand_transport_layer_end.values == 0, initial_sand_mass)
replaced_mass_silt = mass_silt_transport_layer_end.where(mass_silt_transport_layer_end.values == 0, initial_silt_mass)

In [ ]:
# Reset mass of sediment in transport layer at final timestep
trim.MSED[last_timestep, sand, 0] = replaced_mass_sand.values
trim.MSED[last_timestep, silt, 0] = replaced_mass_silt.values

## Write to NetCDF
Write only last timestep to save space

In [ ]:
only_last_timestep = trim.isel(time=-1)

In [ ]:
only_last_timestep.load().to_netcdf(new_write_filename, mode='w', engine='netcdf4', format='NETCDF3_64BIT') 

In [ ]:
trim.time.size

In [ ]:
only_last_timestep.time.size